In [ ]:
# Import all libraries

import numpy as np
from sklearn.cluster import DBSCAN
import pandas as pd

from utilities import df_convoys, filterDF, hb_atom_types_in_data

from hyperparameters import Hyperparameters
import time
from BARMCMiner import BARMCMiner


In [ ]:
data = np.load(r"C:\Users\ashamail\OneDrive - Iowa State University\Molecular Simulation Data\Parsed Data\convoy paper\data\dcd2-55k-65k-fixed.npy")

fileLocation = r"C:\Users\ashamail\OneDrive - Iowa State University\Molecular Simulation Data\Parsed Data\convoy paper\data\allAtomsWithRings.csv" #location for information of atoms
df = pd.read_csv(fileLocation)


nitrogens_and_oxygens_indices = filterDF(df)

atomType = hb_atom_types_in_data(df)


dataSize = [500]
Ks = np.arange(0, 1, 1)
Ms = [25]
t1s= [2]
t2s = [0.25]
starterFrames = [379]
epsilons = [3.5]


for varyk in Ks:
    for varyM in Ms:
        for size in dataSize:
            for t_1 in t1s:
                for t_2 in t2s:
                    for startingPoint in starterFrames:
                        for eps in epsilons:

                            hyperParameters = Hyperparameters(m = varyM, k = varyk, epsilon = eps, t1 = t_1,  t2 = t_2, threshold=4)
                            print(f"K {varyk + 1}, M {varyM}, S {size}, t1 = {hyperParameters.t1}, t2 = {hyperParameters.t2}, epsilon = {hyperParameters.epsilon}, threshold = {hyperParameters.threshold}, DATA = {[startingPoint, startingPoint+size]}")

                            currentData = data[startingPoint:startingPoint + size]

                            transposedData = currentData.transpose((1, 0, 2)).tolist()
                            transposedDataNP = np.array(transposedData)

                            totalTime = 0
                            t1 = time.time()
                            clustering_clf = DBSCAN(eps=hyperParameters.epsilon)

                            clf = BARMCMiner(clustering_clf, k=hyperParameters.k, m=hyperParameters.m, t1=hyperParameters.t1, t2 = hyperParameters.t2, data=currentData, threshold=hyperParameters.threshold, df=df, atomType=atomType, nitrogens_and_oxygens_indices=nitrogens_and_oxygens_indices)

                            convoys = clf.fit_predict(transposedDataNP)

                            sorted_list = sorted(convoys, key=lambda x: x.start_time)
                            totalTime += time.time() - t1

                            display(df_convoys(sorted_list))
                            print("Total time", totalTime)